<a href="https://colab.research.google.com/github/ViMo018/Banana-expiry-predictor/blob/main/BananaShelfLifePredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd

from PIL import Image
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import files



os.makedirs('dataset/train', exist_ok=True)

# Temporary data
for i in range(50):
    random_pixels = np.random.randint(
        0, 255, (224, 224, 3), dtype='uint8'
    )
    img = Image.fromarray(random_pixels)
    img.save(f'dataset/train/banana_{i}.jpg')


# -------------------- Label preparation

filenames = os.listdir('dataset/train')

# Dummy
labels = [np.random.randint(1, 15) for _ in filenames]

df = pd.DataFrame({
    'filename': filenames,
    'days_left': labels
})


# -------------------- Data generators

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=df,
    directory='dataset/train',
    x_col='filename',
    y_col='days_left',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    subset='training'
)

val_generator = train_datagen.flow_from_dataframe(
    dataframe=df,
    directory='dataset/train',
    x_col='filename',
    y_col='days_left',
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    subset='validation'
)


# --------Model

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

# Using pretrained features only
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1)
])

model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)


# -------------------- Training
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)


# -------------------- Save

model.save('banana_model.h5')
files.download('banana_model.h5')


Found 40 validated image filenames.
Found 10 validated image filenames.
Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - loss: 53.2771 - mae: 6.2320 - val_loss: 26.9108 - val_mae: 4.7001
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 897ms/step - loss: 20.1457 - mae: 3.5451 - val_loss: 20.0746 - val_mae: 3.5633
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 926ms/step - loss: 22.1383 - mae: 4.0903 - val_loss: 23.5989 - val_mae: 3.7769
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 20.3357 - mae: 3.7493 - val_loss: 25.1351 - val_mae: 3.9197
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 994ms/step - loss: 18.5980 - mae: 3.6372 - val_loss: 23.2347 - val_mae: 3.7343


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
smodel.save_weights('banana_weights.weights.h5')

from google.colab import files
files.download('banana_weights.weights.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>